### Estudo de Correlação entre Temperatura e Cobertura Vegetal

Este Jupyter Notebook faz parte do módulo de estudo de correlação do projeto PRECORSIA.  
Ele se concentra na análise da correlação entre a temperatura e a cobertura vegetal.

#### Como Utilizar Este Notebook

1. Faça uma cópia deste notebook para o seu ambiente de desenvolvimento.

2. Certifique-se de que você tem todas as dependências necessárias instaladas. Você pode fazê-lo executando o seguinte comando em seu ambiente:

        `pip install numpy pandas matplotlib earthengine-api geemap --upgrade`

3. Execute cada célula do notebook sequencialmente para realizar a análise. Certifique-se de ler e entender cada passo, e ajuste o código, conforme necessário, para atender aos seus requisitos específicos.

4. Se você desejar contribuir com melhorias ou adicionar novas análises, siga as diretrizes no README do sub-módulo para contribuições.  
Você pode criar um novo notebook ou fazer modificações neste, se necessário.

#### Sobre a autorização para utilizar os serviços Google Earth Engine

Sempre que inicializar a aplicação, será solicitado acesso ao serviço GEE através da sua conta google pessoal.  
Para prover acesso e utilizar a API, relize a autorização executando o código abaixo:
```python
import ee
ee.Authenticate()
ee.Initialize()
```
Você será direcionado(a) à uma página WEB, onde será requisitado permissões de acesso.  

> Granting permission. This creates a web application definition controlled by your project provided above. After you click Generate Token, Google will ask for your permission to grant the application access to your data. See details in the step-by-step guide.

> Expiry period. The granted permissions will expire in a week, after which you'll need to call ee.Authenticate() again.

> Revoking permissions. You can view all applications connected to your account, and revoke permissions if needed, on https://myaccount.google.com/permissions. Search for "Earth Engine Notebook Client" to find the application defined by this page.

> Technical details. The web application is defined by a development-mode "OAuth2 Client" on your specified project, which you can manage on the Google Cloud Console.

In [1]:
# Importing all dependencies
import geemap.core as geemap
import ee

In [2]:
# Execute only once per session
ee.Initialize()

In [3]:
# LIMITS
img_list_amount = 10

In [29]:
# Filter region and date to limit queries
START = ee.Date('2023-01-01')
END = START.advance(365, 'day')

filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(20.6729, 52.4305)),
    ee.Filter.date(START, END),
)

In [35]:
# Get a DYNAMICWORLD/V1 dataset's image list
image_collection = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(filter)
image_coll_list = image_collection.toList(100)

# Get the time_start and id propertie
image_time_list = image_coll_list.map(lambda img: ee.Dictionary(
    {'id': ee.Image(img).id(), 'time_start': ee.Image(img).get('system:time_start')})).getInfo()

image_time_list

[{'id': '20230112T095249_20230112T095252_T34UDD', 'time_start': 1673517353066},
 {'id': '20230228T094029_20230228T094031_T34UDD', 'time_start': 1677577559964},
 {'id': '20230228T094029_20230228T094559_T34UDD', 'time_start': 1677577566485},
 {'id': '20230308T095031_20230308T095141_T34UDD', 'time_start': 1678269353065},
 {'id': '20230422T095029_20230422T095031_T34UDD', 'time_start': 1682157359745},
 {'id': '20230424T094031_20230424T094030_T34UDD', 'time_start': 1682329559725},
 {'id': '20230509T093549_20230509T093842_T34UDD', 'time_start': 1683625562832},
 {'id': '20230512T094549_20230512T094549_T34UDD', 'time_start': 1683885360488},
 {'id': '20230514T094031_20230514T094031_T34UDD', 'time_start': 1684057560491},
 {'id': '20230527T095031_20230527T095031_T34UDD', 'time_start': 1685181361280},
 {'id': '20230529T093549_20230529T093840_T34UDD', 'time_start': 1685353564476},
 {'id': '20230601T094549_20230601T094549_T34UDD', 'time_start': 1685613361374},
 {'id': '20230603T094031_20230603T094034

In [36]:
# Get a COPERNICUS/S2 dataset's image list
image_collection = ee.ImageCollection('COPERNICUS/S2').filter(filter)
image_coll_list = image_collection.toList(100)

# Get the time_start and id propertie
image_time_list = image_coll_list.map(lambda img: ee.Dictionary(
    {'id': ee.Image(img).id(), 'time_start': ee.Image(img).get('system:time_start')})).getInfo()

image_time_list

[{'id': '20230102T095319_20230102T095318_T34UDD', 'time_start': 1672653356237},
 {'id': '20230104T094411_20230104T094406_T34UDD', 'time_start': 1672825559217},
 {'id': '20230107T095401_20230107T095402_T34UDD', 'time_start': 1673085356475},
 {'id': '20230109T094259_20230109T094258_T34UDD', 'time_start': 1673257556773},
 {'id': '20230112T095249_20230112T095252_T34UDD', 'time_start': 1673517353066},
 {'id': '20230114T094341_20230114T094343_T34UDD', 'time_start': 1673689557301},
 {'id': '20230117T095331_20230117T095354_T34UDD', 'time_start': 1673949353491},
 {'id': '20230119T094229_20230119T094227_T34UDD', 'time_start': 1674121557622},
 {'id': '20230122T095219_20230122T095219_T34UDD', 'time_start': 1674381355593},
 {'id': '20230124T094301_20230124T094302_T34UDD', 'time_start': 1674553556788},
 {'id': '20230127T095251_20230127T095354_T34UDD', 'time_start': 1674813355213},
 {'id': '20230129T094149_20230129T094143_T34UDD', 'time_start': 1674985557673},
 {'id': '20230201T095129_20230201T095130

### Get interactive map example
by: google in the [link](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1)

```py
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2021-04-02')
END = START.advance(22, 'day')

col_filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(20.6729, 52.4305)),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
m = geemap.Map()
m.set_center(20.6729, 52.4305, 12)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m
```
---  


--- 
### Get image as PixelArray example
by: google in the [link](https://developers.google.com/earth-engine/apidocs/ee-data-getpixels)

```py
# Region of interest.
coords = [
    -121.58626826832939,
    38.059141484827485,
]
region = ee.Geometry.Point(coords)

# Get a Sentinel-2 image.
image = (ee.ImageCollection('COPERNICUS/S2')
  .filterBounds(region)
  .filterDate('2020-04-01', '2020-09-01')
  .sort('CLOUD_COVERAGE_ASSESSMENT')
  .first())
image_id = image.getInfo()['id']

# Make a projection to discover the scale in degrees.
proj = ee.Projection('EPSG:4326').atScale(10).getInfo()

# Get scales out of the transform.
scale_x = proj['transform'][0]
scale_y = -proj['transform'][4]

# Make a request object.
request = {
    'assetId': image_id,
    'fileFormat': 'PNG',
    'bandIds': ['B4', 'B3', 'B2'],
    'grid': {
        'dimensions': {
            'width': 640,
            'height': 640
        },
        'affineTransform': {
            'scaleX': scale_x,
            'shearX': 0,
            'translateX': coords[0],
            'shearY': 0,
            'scaleY': scale_y,
            'translateY': coords[1]
        },
        'crsCode': proj['crs'],
    },
    'visualizationOptions': {'ranges': [{'min': 0, 'max': 3000}]},
}

image_png = ee.data.getPixels(request)
# Do something with the image...
```